In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF


s1_exp1_impressions_df = pd.read_csv("1stSession_EXP1 Impressions.csv")
s1_exp2_impressions_df = pd.read_csv("1stSession_EXP2 Impressions.csv")
s1_exp1_background_df = pd.read_csv("1stSession_EXP1 BackgroundEnvironment.csv")
s1_exp2_background_df = pd.read_csv("1stSession_EXP2 BackgroundEnvironment.csv")


s2_exp1_impressions_df = pd.read_csv("2ndSession_EXP1 Impressions.csv")
s2_exp2_impressions_df = pd.read_csv("2ndSession_EXP2 Impressions.csv")
s2_exp1_background_df = pd.read_csv("2ndSession_EXP1 BackgroundEnvironment.csv")
s2_exp2_background_df = pd.read_csv("2stSession_EXP2 BackgroundEnvironment.csv")

In [2]:
# Combined the data from the 2 experimental sessions

combined_e1 = pd.concat(
    [s1_exp1_impressions_df, s2_exp1_impressions_df], ignore_index=True
)
combined_e2 = pd.concat(
    [s1_exp2_impressions_df, s2_exp2_impressions_df], ignore_index=True
)

In [3]:
# Group by scene number and scene type

groups_e1 = (
    combined_e1.groupby(["Scene Number", "Scene Type"])["Sentences Overall"]
    .agg(lambda x: ";;;\n".join(x))
    .reset_index()
)
groups_e2 = (
    combined_e2.groupby(["Scene Number", "Scene Type"])["Sentences Overall"]
    .agg(lambda x: ";;;\n".join(x))
    .reset_index()
)

In [6]:
def preprocess(grouped_dataframe, tfidf_vectorizer, name):
    """ Find the top 10 words within each group in the dataframe"""
    with open(name, "w") as file:
        file.write("Scene Number, Scene Type, Top words\n")
        for index, row in grouped_dataframe.iterrows():
            tfidf_result = tfidf_vectorizer.fit_transform(
                row["Sentences Overall"].split(";;;\n")
            )
            nmf_result = NMF(n_components=1, random_state=1).fit(tfidf_result)
            feature_names = tfidf_vectorizer.get_feature_names_out()
            for index, topic in enumerate(nmf_result.components_):
                top_words = " ".join(
                    [feature_names[i] for i in topic.argsort()[:-11:-1]]
                )
                file.write(f"{row['Scene Number']}, {row['Scene Type']}, {top_words}\n")

In [7]:
tfidf_vectorizer = TfidfVectorizer(
    max_df=0.95, min_df=2, stop_words="english", ngram_range=(1, 1)
)
preprocess(groups_e1, tfidf_vectorizer, "exp_1.csv")
preprocess(groups_e2, tfidf_vectorizer, "exp_2.csv")